# 100k DOFs case

From the Kratos outputs it seems that on every "Fluid Dynamics Analysis" step 6
problems are solved using AMGCL: 2 larger ones, and 4 smaller ones (a larger
problem has exactly 4 times more DOFs than a smaller one). Riccardo says these
are all physically different problems, which seems to be confirmed by the fact
that each of the problems takes distinctive and repeated across time steps
number of iterations to converge. I'll consider each of these as a separate
sequence of problems.

AMGCL parameters:

- `solver.type=bicgstab`
- `solver.tol=1e-6`
- `precond.coarse_enough=500`
- `precond.coarsening.type=aggregation`
- `precond.relax.type=damped_jacobi`

The larger problems were solved with 4x4 block values both with OpenMP and VexCL(CUDA) backends.

In [1]:
import pandas
import re
from pathlib import Path
def csv(fname):
    return pandas.read_csv(fname, delimiter='\t', header=None)

def speedup(full, part):
    data = {}
    for fname in (full, part):
        for line in open(fname):
            if m := re.search(r'amgcl:\s+(\d+\.\d+) s', line.strip()):
                data[(fname, 'total')] = float(m.group(1))
            elif m := re.search(r'setup:\s+(\d+\.\d+) s', line.strip()):
                data[(fname, 'setup')] = float(m.group(1))
    print(f"Total speedup: {100 * (data.get((full, 'total'), 0) / data.get((part, 'total'), 1) - 1):.2f}%")
    print(f"Setup speedup: {100 * (data.get((full, 'setup'), 0) / data.get((part, 'setup'), 1) - 1):.2f}%")
    
def report(size, device, num, mode):
    full = Path(f'{size}/{device}-{num}-none.time')
    part = Path(f'{size}/{device}-{num}-{mode}.time')
    
    print('*** Full AMGCL rebuild on every step ***\n')
    print(full.read_text().strip())
    log = csv(full.with_suffix('.log'))
    print(f"\nAverage iterations: {log.mean()[2]:.1f}")
    print(f"Full rebuilds: {log.sum()[1]:.0f}")
    
    print('\n\n*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***\n')
    print(part.read_text().strip())
    log = csv(part.with_suffix('.log'))
    print(f"\nAverage iterations: {log.mean()[2]:.1f}")
    print(f"Full rebuilds: {log.sum()[1]:.0f}")
    speedup(full, part)

## Small problem 0, OpenMP

In [2]:
report('small', 'cpu', 0, 'full')

*** Full AMGCL rebuild on every step ***

[Profile:        4.254 s] (100.00%)
[ self:          0.015 s] (  0.35%)
[  amgcl:        3.925 s] ( 92.27%)
[    setup:      1.145 s] ( 26.92%)
[      full:     1.145 s] ( 26.92%)
[    solve:      2.780 s] ( 65.35%)
[  reading:      0.314 s] (  7.38%)

Average iterations: 5.0
Full rebuilds: 49


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:       3.480 s] (100.00%)
[ self:         0.011 s] (  0.32%)
[  amgcl:       3.212 s] ( 92.30%)
[    setup:     0.026 s] (  0.75%)
[    solve:     3.186 s] ( 91.55%)
[  reading:     0.257 s] (  7.38%)

Average iterations: 5.5
Full rebuilds: 1
Total speedup: 22.20%
Setup speedup: 4303.85%


In [3]:
report('small', 'cpu', 0, 'part')

*** Full AMGCL rebuild on every step ***

[Profile:        4.254 s] (100.00%)
[ self:          0.015 s] (  0.35%)
[  amgcl:        3.925 s] ( 92.27%)
[    setup:      1.145 s] ( 26.92%)
[      full:     1.145 s] ( 26.92%)
[    solve:      2.780 s] ( 65.35%)
[  reading:      0.314 s] (  7.38%)

Average iterations: 5.0
Full rebuilds: 49


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:           3.793 s] (100.00%)
[ self:             0.061 s] (  1.62%)
[  amgcl:           3.472 s] ( 91.54%)
[    setup:         0.703 s] ( 18.53%)
[      full:        0.118 s] (  3.11%)
[      partial:     0.585 s] ( 15.43%)
[    solve:         2.769 s] ( 73.00%)
[  reading:         0.260 s] (  6.84%)

Average iterations: 5.0
Full rebuilds: 5
Total speedup: 13.05%
Setup speedup: 62.87%


## Small problem 0, VexCL(CUDA)

In [4]:
report('small', 'gpu', 0, 'full')

*** Full AMGCL rebuild on every step ***

[Profile:        2.940 s] (100.00%)
[ self:          0.008 s] (  0.28%)
[  amgcl:        2.671 s] ( 90.85%)
[    setup:      1.890 s] ( 64.29%)
[      full:     1.890 s] ( 64.29%)
[    solve:      0.781 s] ( 26.56%)
[  reading:      0.261 s] (  8.87%)

Average iterations: 5.0
Full rebuilds: 49


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:       1.678 s] (100.00%)
[ self:         0.487 s] ( 29.03%)
[  amgcl:       0.936 s] ( 55.77%)
[    setup:     0.039 s] (  2.32%)
[    solve:     0.897 s] ( 53.45%)
[  reading:     0.255 s] ( 15.20%)

Average iterations: 5.5
Full rebuilds: 1
Total speedup: 185.36%
Setup speedup: 4746.15%


In [5]:
report('small', 'gpu', 0, 'part')

*** Full AMGCL rebuild on every step ***

[Profile:        2.940 s] (100.00%)
[ self:          0.008 s] (  0.28%)
[  amgcl:        2.671 s] ( 90.85%)
[    setup:      1.890 s] ( 64.29%)
[      full:     1.890 s] ( 64.29%)
[    solve:      0.781 s] ( 26.56%)
[  reading:      0.261 s] (  8.87%)

Average iterations: 5.0
Full rebuilds: 49


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:           2.334 s] (100.00%)
[ self:             0.008 s] (  0.33%)
[  amgcl:           2.065 s] ( 88.48%)
[    setup:         1.284 s] ( 55.02%)
[      full:        0.189 s] (  8.09%)
[      partial:     1.096 s] ( 46.93%)
[    solve:         0.781 s] ( 33.46%)
[  reading:         0.261 s] ( 11.18%)

Average iterations: 5.0
Full rebuilds: 5
Total speedup: 29.35%
Setup speedup: 47.20%


## Small problem 2, OpenMP

In [6]:
report('small', 'cpu', 2, 'full')

*** Full AMGCL rebuild on every step ***

[Profile:       11.078 s] (100.00%)
[ self:          0.066 s] (  0.60%)
[  amgcl:       10.753 s] ( 97.07%)
[    setup:      1.246 s] ( 11.24%)
[      full:     1.246 s] ( 11.24%)
[    solve:      9.508 s] ( 85.82%)
[  reading:      0.258 s] (  2.33%)

Average iterations: 15.7
Full rebuilds: 49


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:      11.897 s] (100.00%)
[ self:         0.066 s] (  0.56%)
[  amgcl:      11.569 s] ( 97.25%)
[    setup:     0.245 s] (  2.06%)
[    solve:    11.324 s] ( 95.19%)
[  reading:     0.261 s] (  2.19%)

Average iterations: 18.1
Full rebuilds: 9
Total speedup: -7.05%
Setup speedup: 408.57%


In [7]:
report('small', 'cpu', 2, 'part')

*** Full AMGCL rebuild on every step ***

[Profile:       11.078 s] (100.00%)
[ self:          0.066 s] (  0.60%)
[  amgcl:       10.753 s] ( 97.07%)
[    setup:      1.246 s] ( 11.24%)
[      full:     1.246 s] ( 11.24%)
[    solve:      9.508 s] ( 85.82%)
[  reading:      0.258 s] (  2.33%)

Average iterations: 15.7
Full rebuilds: 49


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:          10.380 s] (100.00%)
[ self:             0.062 s] (  0.59%)
[  amgcl:          10.057 s] ( 96.89%)
[    setup:         0.749 s] (  7.22%)
[      full:        0.128 s] (  1.23%)
[      partial:     0.622 s] (  5.99%)
[    solve:         9.308 s] ( 89.67%)
[  reading:         0.261 s] (  2.51%)

Average iterations: 15.6
Full rebuilds: 5
Total speedup: 6.92%
Setup speedup: 66.36%


## Small problem 2, VexCL(CUDA)

In [8]:
report('small', 'gpu', 2, 'full')

*** Full AMGCL rebuild on every step ***

[Profile:        4.811 s] (100.00%)
[ self:          0.008 s] (  0.16%)
[  amgcl:        4.540 s] ( 94.35%)
[    setup:      1.904 s] ( 39.57%)
[      full:     1.904 s] ( 39.57%)
[    solve:      2.636 s] ( 54.78%)
[  reading:      0.264 s] (  5.48%)

Average iterations: 15.7
Full rebuilds: 49


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:       4.136 s] (100.00%)
[ self:         0.478 s] ( 11.56%)
[  amgcl:       3.404 s] ( 82.30%)
[    setup:     0.355 s] (  8.59%)
[    solve:     3.049 s] ( 73.71%)
[  reading:     0.254 s] (  6.14%)

Average iterations: 18.0
Full rebuilds: 9
Total speedup: 33.37%
Setup speedup: 436.34%


In [9]:
report('small', 'gpu', 2, 'part')

*** Full AMGCL rebuild on every step ***

[Profile:        4.811 s] (100.00%)
[ self:          0.008 s] (  0.16%)
[  amgcl:        4.540 s] ( 94.35%)
[    setup:      1.904 s] ( 39.57%)
[      full:     1.904 s] ( 39.57%)
[    solve:      2.636 s] ( 54.78%)
[  reading:      0.264 s] (  5.48%)

Average iterations: 15.7
Full rebuilds: 49


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:           4.225 s] (100.00%)
[ self:             0.008 s] (  0.18%)
[  amgcl:           3.953 s] ( 93.55%)
[    setup:         1.340 s] ( 31.70%)
[      full:        0.205 s] (  4.85%)
[      partial:     1.135 s] ( 26.85%)
[    solve:         2.613 s] ( 61.84%)
[  reading:         0.265 s] (  6.27%)

Average iterations: 15.6
Full rebuilds: 5
Total speedup: 14.85%
Setup speedup: 42.09%


## Large problem 0, OpenMP

In [10]:
report('large', 'cpu', 0, 'full')

*** Full AMGCL rebuild on every step ***

[Profile:      298.574 s] (100.00%)
[  amgcl:       78.979 s] ( 26.45%)
[    setup:      7.926 s] (  2.65%)
[      full:     7.926 s] (  2.65%)
[    solve:     71.052 s] ( 23.80%)
[  reading:    219.492 s] ( 73.51%)

Average iterations: 16.7
Full rebuilds: 49


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:      472.871 s] (100.00%)
[  amgcl:      243.182 s] ( 51.43%)
[    setup:      3.859 s] (  0.82%)
[      full:     3.859 s] (  0.82%)
[    solve:    239.322 s] ( 50.61%)
[  reading:    229.582 s] ( 48.55%)

Average iterations: 45.6
Full rebuilds: 25
Total speedup: -67.52%
Setup speedup: 105.39%


In [11]:
report('large', 'cpu', 0, 'part')

*** Full AMGCL rebuild on every step ***

[Profile:      298.574 s] (100.00%)
[  amgcl:       78.979 s] ( 26.45%)
[    setup:      7.926 s] (  2.65%)
[      full:     7.926 s] (  2.65%)
[    solve:     71.052 s] ( 23.80%)
[  reading:    219.492 s] ( 73.51%)

Average iterations: 16.7
Full rebuilds: 49


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:         305.686 s] (100.00%)
[  amgcl:          77.774 s] ( 25.44%)
[    setup:         6.011 s] (  1.97%)
[      full:        0.769 s] (  0.25%)
[      partial:     5.242 s] (  1.71%)
[    solve:        71.763 s] ( 23.48%)
[  reading:       227.820 s] ( 74.53%)

Average iterations: 16.9
Full rebuilds: 5
Total speedup: 1.55%
Setup speedup: 31.86%


## Large problem 0, VexCL(CUDA)

In [12]:
report('large', 'gpu', 0, 'full')

*** Full AMGCL rebuild on every step ***

[Profile:      266.945 s] (100.00%)
[  amgcl:       35.186 s] ( 13.18%)
[    setup:     13.789 s] (  5.17%)
[      full:    13.789 s] (  5.17%)
[    solve:     21.398 s] (  8.02%)
[  reading:    231.661 s] ( 86.78%)

Average iterations: 16.7
Full rebuilds: 49


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:      292.361 s] (100.00%)
[ self:          7.125 s] (  2.44%)
[  amgcl:       66.063 s] ( 22.60%)
[    setup:      6.863 s] (  2.35%)
[      full:     6.863 s] (  2.35%)
[    solve:     59.200 s] ( 20.25%)
[  reading:    219.173 s] ( 74.97%)

Average iterations: 46.1
Full rebuilds: 25
Total speedup: -46.74%
Setup speedup: 100.92%


In [13]:
report('large', 'gpu', 0, 'part')

*** Full AMGCL rebuild on every step ***

[Profile:      266.945 s] (100.00%)
[  amgcl:       35.186 s] ( 13.18%)
[    setup:     13.789 s] (  5.17%)
[      full:    13.789 s] (  5.17%)
[    solve:     21.398 s] (  8.02%)
[  reading:    231.661 s] ( 86.78%)

Average iterations: 16.7
Full rebuilds: 49


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:         257.506 s] (100.00%)
[  amgcl:          32.807 s] ( 12.74%)
[    setup:        11.209 s] (  4.35%)
[      full:        1.475 s] (  0.57%)
[      partial:     9.735 s] (  3.78%)
[    solve:        21.597 s] (  8.39%)
[  reading:       224.614 s] ( 87.23%)

Average iterations: 16.9
Full rebuilds: 5
Total speedup: 7.25%
Setup speedup: 23.02%
